In [1]:
[global]
# Working directory: change accordingly
parameter: cwd = path
# Genotype file in plink binary format
parameter: bfile = path
# Path to bgen files
parameter: bgenFile = paths
# Path to sample file
parameter: sampleFile = path
# Path to summary stats file
parameter: sumstatsFile = path
# Path to samples of unrelated individuals
parameter: unrelated_samples = path
# Number of samples to use to compute LD
parameter: ld_sample_size = 1000
# Clumping parameteres
parameter: clump_field = str
parameter: clump_annotate = str
parameter: clump_p1 = 5e-08
parameter: clump_p2 = 1
# r2 = 0.04 => r = 0.2
parameter: clump_r2 = 0.04
parameter: clump_kb = 2000
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Output the bgen file with 8bit formatting
#parameter: bgen_bits=16
# Specific number of threads to use
parameter: numThreads = int
# Load specific modules for each step
parameter: qctool_module = '''
module load QCTOOL/2.0-foss-2016b-rc7-CentOS6.8
echo "Module qctool loaded"
{cmd}
'''
parameter: plink2_module = '''
module load PLINK/2_x86_64_20180428
echo "Module plink2 loaded"
{cmd}
'''

parameter: plink_module = '''
module load PLINK/1.90-beta5.3
echo "Module plink loaded"
{cmd}
'''

In [ ]:
# Select a subset of samples from the BGEN files
[default_1]
depends: f'{cwd}/{unrelated_samples:bn}.{ld_sample_size}.txt'
input: bgenFile, group_by=1
output: f'{cwd}/{_input:bn}.{ld_sample_size}.bgen', f'{cwd}/{_input:bn}.{ld_sample_size}.sample'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h',  mem = '12G', tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', template = '{cmd}' if executable('qctools').target_exists() else qctool_module
    qctool \
    -g ${_input} \
    -s ${sampleFile} \
    -og ${_output[0]} \
    -os ${_output[1]} \
    -incl-samples ${_depends}

In [ ]:
[default_2]
output: f'{_input[0]:n}.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '12G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', template = '{cmd}' if executable('plink2').target_exists() else plink2_module
    plink2 \
    --bgen ${_input[0]} ref-first \
    --sample ${_input[1]} \
    --make-bed \
    --out ${_output:n} \
    --threads ${numThreads}

In [ ]:
# Extract snps only and remove multiallelic places
[default_3]
output: f'{cwd}/{_input[0]:bn}.snps_only.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '12G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', template = '{cmd}' if executable('plink').target_exists() else plink_module
    cut  -d$'\t' -f2 ${_input[0]:n}.bim | sort | uniq -d > ${_output:nn}.exclude
    plink \
    --bfile ${_input[0]:n} \
    --exclude ${_output:nn}.exclude \
    --make-bed \
    --out ${_output:n} \
    --threads ${numThreads} \
    --memory 12000

In [ ]:
# Merge all the .bed files into one reference file 
[default_4]
input: group_by = 'all'
output: f'{cwd}/{sumstatsFile:bn}.ref_geno.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '64G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', template_name='conda', env_name='plink1.7'
    echo -e ${' '.join([str(x)[:-4] for x in _input[1:]])} | sed 's/ /\n/g' > ${_output:n}.merge_list
    plink \
    --bfile ${_input[0]:n} \
    --merge-list ${_output:n}.merge_list \
    --make-bed \
    --out ${_output:n} \
    --threads ${numThreads} \
    --memory 64000 \
    --noweb

In [ ]:
# Perform LD-clumping in PLINKv1.9
[default_5]
output: f'{_input:nn}.clumped', f'{_input:nn}.clumped_region'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '12G',cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', template = '{cmd}' if executable('plink').target_exists() else plink_module    
    plink \
    --bfile ${_input:n} \
    --clump ${sumstatsFile} \
    --clump-field ${clump_field} \
    --clump-p1 ${clump_p1} \
    --clump-p2 ${clump_p2} \
    --clump-r2 ${clump_r2} \
    --clump-kb ${clump_kb} \
    --clump-verbose \
    --clump-annotate ${clump_annotate} \
    --out ${_output[0]:n} \
    --threads ${numThreads} \
    && touch ${_output[0]} # need to touch and create empty file because some chroms may not have anything significant to clump.
    grep "RANGE" ${_output[0]} | awk -F ":" '{print $2, $3}' | sort -V | sed 's/\../ /g; s/^[[:blank:]]*//g' > ${_output[1]}